In [ ]:
!pip install pytorch_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/176.4 kB ? eta -:--:--
     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/176.4 kB 1.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 122.9/176.4 kB 1.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
import string
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from transformers import pipeline
# from transformers import TrainingArguments
import nltk

import torch
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split
# from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
# from pytorch_transformers import AdamW

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Before Truncation Part

In [ ]:
train_dataset = pd.read_csv("/content/drive/MyDrive/Genre_Dataset/train_genre.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
train_dataset.columns

Index(['Artist', 'Song', 'Genre', 'Language', 'Lyrics'], dtype='object')

# Data truncation

In [ ]:
genre_list = train_dataset.groupby('Genre')['Lyrics'].count().index.to_list()

In [ ]:
genre_list[0]

'Country'

In [ ]:
indexes = train_dataset.groupby('Genre')['Lyrics'].groups[genre_list[0]]

In [ ]:
import random

In [ ]:
train_dataset_new = pd.DataFrame()
train_dataset_new['Artist'] = None
train_dataset_new['Song'] = None
train_dataset_new['Genre'] = None
train_dataset_new['Language'] = None
train_dataset_new['Lyrics'] = None

In [ ]:
Artist = []
Song = []
Genre = []
Language = []
Lyrics = []
for i in genre_list:
  indexes = train_dataset.groupby('Genre')['Lyrics'].groups[i]
  indexes = indexes.to_list()
  if len(indexes)<4000:
    for j in range(len(indexes)):
      Artist.append(train_dataset['Artist'].loc[indexes[j]])
      Song.append(train_dataset['Song'].loc[indexes[j]])
      Genre.append(train_dataset['Genre'].loc[indexes[j]])
      Language.append(train_dataset['Language'].loc[indexes[j]])
      Lyrics.append(train_dataset['Lyrics'].loc[indexes[j]])
  else:
    indexes = random.sample(indexes,6000)
    for j in range(len(indexes)):
      Artist.append(train_dataset['Artist'].loc[indexes[j]])
      Song.append(train_dataset['Song'].loc[indexes[j]])
      Genre.append(train_dataset['Genre'].loc[indexes[j]])
      Language.append(train_dataset['Language'].loc[indexes[j]])
      Lyrics.append(train_dataset['Lyrics'].loc[indexes[j]])

In [ ]:
train_dataset_new['Artist'] = Artist
train_dataset_new['Song'] = Song
train_dataset_new['Genre'] = Genre
train_dataset_new['Language'] = Language
train_dataset_new['Lyrics'] = Lyrics

In [ ]:
train_dataset_new

,Artist,Song,Genre,Language,Lyrics
0,brad paisley,be the lake,Country,en,there aint nothing that i would rather see tha...
1,gene watson,sometimes i get lucky and forget,Country,en,i stay out late most every night until closing...
2,charlie landsborough,long and heavy chain,Country,en,your hunger after money your parents agree the...
3,be good tanyas,rain and snow,Country,en,well i married me a wife she gave me trouble a...
4,buck owens,ashes of love,Country,en,ashes of love cold as ice you made the debt il...
...,...,...,...,...,...
44804,echo & the bunnymen,evergreen,Rock,en,theres no more wishes in the well no more drea...
44805,journey,"mother, father",Rock,en,she sits alone an empty stare a mothers face s...
44806,xtc,science friction,Rock,en,i look out of my window at night i see the sta...
44807,wilco,art of almost,Rock,en,i cant be so far away from my wasteland ill ne...


In [ ]:
train_dataset_new.groupby('Genre')['Lyrics'].count()

Genre
Country       1890
Electronic    1993
Folk          6000
Hip-Hop       2238
Indie         6000
Jazz          6000
Metal         6000
Pop           6000
R&B           2688
Rock          6000
Name: Lyrics, dtype: int64

In [ ]:
train_dataset_new.to_csv('/content/drive/MyDrive/Genre_Dataset/train_dataset_truncated.csv')

# Creating embeddings using zeugma

In [ ]:
train_dataset = pd.read_csv('/content/drive/MyDrive/Genre_Dataset/train_dataset_truncated.csv')
test_dataset = pd.read_csv("/content/drive/MyDrive/Genre_Dataset/test_genre.csv")

In [ ]:
!pip install zeugma

  Preparing metadata (setup.py) ... done
  Created wheel for zeugma: filename=zeugma-0.49-py3-none-any.whl size=8790 sha256=183e34c8eae78d2433021f4c6fb8e8ebaa7125e4b828f72ae38efcf4d1ab2dd8
  Stored in directory: /root/.cache/pip/wheels/33/55/16/3d9155ec34c3a4b3c5488ea8905fd994a6cafd68b8e3e26489
Successfully built zeugma


In [ ]:
from zeugma.embeddings import EmbeddingTransformer
glove = EmbeddingTransformer('word2vec')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
embeddings_train = glove.transform(train_dataset['Lyrics'])

In [ ]:
embeddings_test = glove.transform(test_dataset['Lyrics'])

In [ ]:
import h5py

In [ ]:
with h5py.File('train_embeddings_word2vec.h5', 'w') as hf:
    hf.create_dataset('embeddings_train', data=embeddings_train)

In [ ]:
with h5py.File('test_embeddings_word2vec.h5', 'w') as hf:
    hf.create_dataset('embeddings_test', data=embeddings_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
labels = train_dataset['Genre'].to_numpy()
labels = labels.reshape(-1, 1)
one_hot_labels = encoder.fit_transform(labels)
one_hot_labels_train = np.argmax(one_hot_labels, axis=1)
labels_test = test_dataset['Genre'].to_numpy()
labels_test = labels_test.reshape(-1, 1)
one_hot_labels_test = encoder.fit_transform(labels_test)
one_hot_labels_test = np.argmax(one_hot_labels_test, axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Creating embeddings using HuggingFace

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("jinaai/jina-embeddings-v2-base-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings_train = model.encode(train_dataset['Lyrics'])

In [ ]:
embeddings_test = model.encode(test_dataset['Lyrics'])

In [ ]:
with h5py.File('train_embeddings_HF.h5', 'w') as hf:
    hf.create_dataset('embeddings_train', data=embeddings_train)

In [ ]:
with h5py.File('test_embeddings_HF.h5', 'w') as hf:
    hf.create_dataset('embeddings_test', data=embeddings_test)

# Using literal eval

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings_train = sbert_model.encode(train_dataset['Lyrics'])

In [ ]:
embeddings_test = sbert_model.encode(test_dataset['Lyrics'])

In [ ]:
import h5py
with h5py.File('train_embeddings_SBERT.h5', 'w') as hf:
    hf.create_dataset('embeddings_train', data=embeddings_train)
with h5py.File('test_embeddings_SBERT.h5', 'w') as hf:
    hf.create_dataset('embeddings_test', data=embeddings_test)

# Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
clf = MultinomialNB()

In [ ]:
scaler = MinMaxScaler()
embeddings_train_scaled = scaler.fit_transform(embeddings_train)

In [ ]:
clf.fit(embeddings_train_scaled,one_hot_labels_train)

MultinomialNB()

In [ ]:
embeddings_test_scaled = scaler.fit_transform(embeddings_test)

In [ ]:
predicted_MNB = clf.predict(embeddings_test_scaled)

In [ ]:
accuracy_score(one_hot_labels_test,predicted_MNB)

0.29023314429741653

In [ ]:
f1_score(one_hot_labels_test,predicted_MNB, average='macro')

0.22768287188692105

In [ ]:
recall_score(one_hot_labels_test,predicted_MNB, average='macro')

0.28818996053312385

In [ ]:
precision_score(one_hot_labels_test,predicted_MNB,average='macro')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.22732783887640678

# Gaussian NB

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(embeddings_train_scaled,one_hot_labels_train)

GaussianNB()

In [ ]:
predicted_GNB = clf.predict(embeddings_test_scaled)

In [ ]:
accuracy_score(one_hot_labels_test,predicted_GNB)

0.3136735979836169

In [ ]:
f1_score(one_hot_labels_test,predicted_GNB, average='macro')

0.2907525176614905

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf = KNeighborsClassifier(5)

In [ ]:
clf.fit(embeddings_train_scaled,one_hot_labels_train)

KNeighborsClassifier()

In [ ]:
predicted_KNN = clf.predict(embeddings_test_scaled)

In [ ]:
accuracy_score(one_hot_labels_test,predicted_KNN)

0.2980466288594833

# XGBOOST

In [ ]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.9 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(embeddings_train_scaled,one_hot_labels_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
predicted_XGB = model.predict(embeddings_test_scaled)

In [ ]:
accuracy_score(one_hot_labels_test,predicted_XGB)

0.3045998739760554

# Develop transformer

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings_train = model.encode(train_dataset['Lyrics'])

In [ ]:
embeddings_test = model.encode(test_dataset['Lyrics'])

In [ ]:
import h5py
with h5py.File('train_embeddings_SBERT_mpnet.h5', 'w') as hf:
    hf.create_dataset('embeddings_train', data=embeddings_train)
with h5py.File('test_embeddings_SBERT_mpnet.h5', 'w') as hf:
    hf.create_dataset('embeddings_test', data=embeddings_test)

# RoBerta

In [69]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tokenizers
from transformers import RobertaTokenizer,TFRobertaModel
from transformers import AutoTokenizer, RobertaForSequenceClassification
from transformers import pipeline
from transformers import TrainingArguments
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_dataset = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/nlp/Genre-Final/train_dataset_truncated.csv')
test_dataset = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/nlp/Genre-Final/test_genre.csv')

In [4]:
train_dataset

,Unnamed: 0,Artist,Song,Genre,Language,Lyrics
0,0,brad paisley,be the lake,Country,en,there aint nothing that i would rather see tha...
1,1,gene watson,sometimes i get lucky and forget,Country,en,i stay out late most every night until closing...
2,2,charlie landsborough,long and heavy chain,Country,en,your hunger after money your parents agree the...
3,3,be good tanyas,rain and snow,Country,en,well i married me a wife she gave me trouble a...
4,4,buck owens,ashes of love,Country,en,ashes of love cold as ice you made the debt il...
...,...,...,...,...,...,...
44804,44804,echo & the bunnymen,evergreen,Rock,en,theres no more wishes in the well no more drea...
44805,44805,journey,"mother, father",Rock,en,she sits alone an empty stare a mothers face s...
44806,44806,xtc,science friction,Rock,en,i look out of my window at night i see the sta...
44807,44807,wilco,art of almost,Rock,en,i cant be so far away from my wasteland ill ne...


In [5]:
train_dataset = train_dataset.sample(frac = 1)
train_dataset = train_dataset.drop(['Unnamed: 0','Artist','Song','Language'],axis=1)
test_dataset = test_dataset.drop(['Artist','Song','Song year','Track_id'],axis=1)

In [6]:
X_train = train_dataset[['Lyrics']].to_numpy().reshape(-1)
y_train = train_dataset[['Genre']].to_numpy().reshape(-1)
X_test = test_dataset[['Lyrics']].to_numpy().reshape(-1)
y_test = test_dataset[['Genre']].to_numpy().reshape(-1)

In [7]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)

        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])

        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN

        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')

        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

        return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [8]:
# Transform categories into numbers
category_to_id = {}
category_to_name_train = {}

for index, c in enumerate(y_train):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name_train[category_id] = c

    y_train[index] = category_id

# Display dictionary
category_to_name_train

{0: 'Indie',
 1: 'Country',
 2: 'Jazz',
 3: 'Folk',
 4: 'Rock',
 5: 'Hip-Hop',
 6: 'Pop',
 7: 'Metal',
 8: 'Electronic',
 9: 'R&B'}

In [9]:
# Transform categories into numbers
category_to_id = {}
category_to_name_test = {}

for index, c in enumerate(y_test):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name_test[category_id] = c

    y_test[index] = category_id

# Display dictionary
category_to_name_test

{0: 'Hip-Hop',
 1: 'Indie',
 2: 'Metal',
 3: 'Pop',
 4: 'Country',
 5: 'Jazz',
 6: 'Rock',
 7: 'R&B',
 8: 'Electronic',
 9: 'Folk'}

In [10]:
MODEL_NAME = 'roberta-base'
MAX_LEN = 1000

In [11]:
categories = train_dataset['Genre'].unique()
n_categories = len(categories)

In [12]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [13]:
X_train = roberta_encode(X_train, tokenizer)
X_test = roberta_encode(X_test, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [65]:
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print('Number of replicas:', strategy.num_replicas_in_sync)

In [78]:
def build_model(n_categories):
  input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
  input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
  input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

  # Import RoBERTa model from HuggingFace
  roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)
  x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

  # Huggingface transformers have multiple outputs, embeddings are the first one,
  # so let's slice out the first position
  x = x[0]

  x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)
  model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(lr=1e-4),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

  return model

In [79]:
BATCH_SIZE = 8

In [81]:
model = build_model(n_categories)
model.summary()

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
print('Training...')
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(X_train,
                    y_train,
                    epochs=3,
                    batch_size=BATCH_SIZE,
                    verbose=1,
                    validation_data=(X_test, y_test))

Training...
Epoch 1/3


701/701 [==============================] - 374s 404ms/step - loss: 4.5259 - accuracy: 0.1328 - val_loss: 2.3322 - val_accuracy: 0.0832
Epoch 2/3
701/701 [==============================] - 222s 317ms/step - loss: 2.2189 - accuracy: 0.1324 - val_loss: 2.3807 - val_accuracy: 0.1021
Epoch 3/3
701/701 [==============================] - 223s 318ms/step - loss: 2.2078 - accuracy: 0.1330 - val_loss: 2.4086 - val_accuracy: 0.1021


In [ ]:
import h5py
model.save("roberta_genre_model.h5")
# model_h5_file = "roberta_genre_model.h5"

# with open(model_h5_file, 'wb') as file:
#     pickle.dump(model, file)
# with h5py.File('model_roberta.h5', 'w') as hf:
#     hf.create_dataset('embeddings_train', data=embeddings_train)

# DistilBERT

In [1]:
# import tensorflow as tf
# import tensorflow.keras.backend as K
import tokenizers
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
import pandas as pd
train_dataset = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/nlp/Genre-Final/train_dataset_truncated.csv')
# test_dataset = pd.read_csv('test_genre.csv')

<IPython.core.display.Javascript object>

In [3]:
train_dataset['Genre'].unique()

array(['Country', 'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz',
       'Metal', 'Pop', 'R&B', 'Rock'], dtype=object)

In [6]:
train_dataset = train_dataset.sample(frac = 1)
# train_dataset = train_dataset.drop(['Unnamed: 0','Artist','Song','Language'],axis=1)
# test_dataset = test_dataset.drop(['Artist','Song','Song year','Track_id'],axis=1)

In [8]:
# X_train = train_dataset['Lyrics'].to_list()
y_train = train_dataset['Genre'].to_numpy().reshape(-1)
# X_test = test_dataset['Lyrics'].to_list()
# y_test = test_dataset['Genre'].to_numpy().reshape(-1)

In [17]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit the encoder to the labels and transform the labels to numerical values
label_encoder.fit_transform(y_train)
custom_labels = ['Country', 'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz',
       'Metal', 'Pop', 'R&B', 'Rock']

# Print out the mapping
for encoded_value, label in enumerate(custom_labels):
    print(f"{label}: {encoded_value}")

Country: 0
Electronic: 1
Folk: 2
Hip-Hop: 3
Indie: 4
Jazz: 5
Metal: 6
Pop: 7
R&B: 8
Rock: 9


In [10]:
y_train

array([9, 2, 7, ..., 4, 9, 9])

In [18]:
train_dataset['Genre'] = y_train
test_dataset['Genre'] = y_test

In [19]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [20]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data.Lyrics[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Genre[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [21]:

train_size = 0.8

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

In [22]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [23]:
import torch
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768,10)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [24]:
import torch.nn as nn
model = DistillBERTClass()
model = nn.DataParallel(model)
model.to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DataParallel(
  (module): DistillBERTClass(
    (l1): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dr

In [25]:
weights = torch.load('DistilBERT-checkpt.pth')
model.load_state_dict(weights)


<All keys matched successfully>

In [26]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [27]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [28]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(6):
    train(epoch)

Training Loss per 5000 steps: 0.0638701394200325
Training Accuracy per 5000 steps: 100.0
Training Loss per 5000 steps: 0.2865028747694639
Training Accuracy per 5000 steps: 90.69186162767447
Training Loss per 5000 steps: 0.3120262205653516
Training Accuracy per 5000 steps: 89.8960103989601
The Total Accuracy for Epoch 0: 89.81900957396951
Training Loss Epoch: 0.31434162950939326
Training Accuracy Epoch: 89.81900957396951
Training Loss per 5000 steps: 0.5035730600357056
Training Accuracy per 5000 steps: 75.0
Training Loss per 5000 steps: 0.23185984051184477
Training Accuracy per 5000 steps: 92.60647870425915
Training Loss per 5000 steps: 0.2513445457103813
Training Accuracy per 5000 steps: 91.93330666933306
The Total Accuracy for Epoch 1: 91.8900220937758
Training Loss Epoch: 0.2529309508653628
Training Accuracy Epoch: 91.8900220937758
Training Loss per 5000 steps: 0.010653601959347725
Training Accuracy per 5000 steps: 100.0
Training Loss per 5000 steps: 0.20156387343792148
Training Accu

In [ ]:
torch.save(model.state_dict(), 'DistilBERT-Final.pth')

In [ ]:
for epoch in range(10):
    train(epoch)

Training Loss per 5000 steps: 1.0070712566375732
Training Accuracy per 5000 steps: 50.0
Training Loss per 5000 steps: 1.0327204129308718
Training Accuracy per 5000 steps: 65.08198360327934
Training Loss per 5000 steps: 1.029923122983179
Training Accuracy per 5000 steps: 65.07099290070992
The Total Accuracy for Epoch 0: 64.99140797607623
Training Loss Epoch: 1.0323886736982368
Training Accuracy Epoch: 64.99140797607623
Training Loss per 5000 steps: 1.0213323831558228
Training Accuracy per 5000 steps: 75.0
Training Loss per 5000 steps: 0.8101512320331113
Training Accuracy per 5000 steps: 73.10537892421516
Training Loss per 5000 steps: 0.8260961693012505
Training Accuracy per 5000 steps: 72.41525847415258
The Total Accuracy for Epoch 1: 72.36939007788614
Training Loss Epoch: 0.8257232330641237
Training Accuracy Epoch: 72.36939007788614
Training Loss per 5000 steps: 0.1048661321401596
Training Accuracy per 5000 steps: 100.0
Training Loss per 5000 steps: 0.617733402360008
Training Accuracy 

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'DistilBERT-Final.pth')